In [1]:
import streamlit as st
# Import Streamlit to build the web app UI and widgets.
import pickle
# Import pickle to load the previously trained ML model saved as a .pkl file.
import numpy as np
# Import NumPy for array creation and numerical operations (model input needs numpy array).
import pandas as pd
# Import Pandas to build a small table to display the entered input values back to the user.
import matplotlib.pyplot as plt
# Import Matplotlib to build the pie chart visualization and then show it with st.pyplot.

# Load Trained Model
model = pickle.load(open("Bankruptcy_Model_Final.pkl", "rb"))
st.set_page_config(
# Configure basic page options like title, icon, and layout for a nicer presentation in Streamlit.

    page_title="Bankruptcy Prediction App",
    # The browser tab title and the app title shown in the top-left of the Streamlit app.
    page_icon="💰",
    # Small emoji icon for the tab and top-left of the app.
    layout="wide"
    # Use a wide layout for more horizontal space (useful for tables/charts).
)

# Sidebar Navigation
st.sidebar.title("📌 Navigation")
# Add a title to the sidebar to tell users this is the navigation area.

menu = st.sidebar.radio("Go to:", ["Home", "Predict", "Project Info", "About Developer"])
# Create a radio button group in the sidebar for navigating between pages; captures the selected option.

# HOME PAGE
if menu == "Home":
    # If the user selected "Home", render the home page content.

    st.title("🏦 Bankruptcy Prediction System")
    # Main heading for the home page.

    st.markdown("""
    ### 📍 Objective  
    Predict whether a company might **go bankrupt** based on financial risk factors.  
    Provide reliable early warnings for better business decisions.

    ✅ **Machine Learning Model Used**: Logistic Regression  
    ✅ **Input Values**: 0 (Low), 0.5 (Medium), 1 (High)

    ---
    """)
    # Markdown block describing the objective, model, and allowed input values.
    # Markdown supports headings, bold, bullet lists and a horizontal rule (---).

    st.image("https://cdn-icons-png.flaticon.com/512/2721/2721298.png", width=250)
    # Display an illustrative image via URL; the width parameter sizes the image.
    # Note: external images require internet access when the app loads.

# ----------------------------
# PREDICTION PAGE
# ----------------------------
elif menu == "Predict":
    # If "Predict" is selected, show the prediction form and result UI.

    st.title("📊 Bankruptcy Prediction")
    # Header for the prediction page.

    with st.form("input_form"):
        # Start a Streamlit form so inputs are submitted together when the user clicks the submit button.

        st.write("### Enter Financial Risk Levels")
        # Small subtitle/instruction for the form.

        col1, col2 = st.columns(2)
        # Create two columns to place sliders side-by-side for a compact layout.

        with col1:
            # Place the next three sliders inside the first column.

            industrial_risk = st.slider("Industrial Risk", 0.0, 1.0, step=0.5)
            # Slider for Industrial Risk; restricts values to 0, 0.5, 1 with step=0.5.
            # The returned value is a float.

            financial_flexibility = st.slider("Financial Flexibility", 0.0, 1.0, step=0.5)
            # Slider for Financial Flexibility; same allowed values as above.

            competitiveness = st.slider("Competitiveness", 0.0, 1.0, step=0.5)
            # Slider for Competitiveness; same allowed values.

        with col2:
            # Place the next three sliders inside the second column.

            management_risk = st.slider("Management Risk", 0.0, 1.0, step=0.5)
            # Slider for Management Risk.

            credibility = st.slider("Credibility", 0.0, 1.0, step=0.5)
            # Slider for Credibility.

            operating_risk = st.slider("Operating Risk", 0.0, 1.0, step=0.5)
            # Slider for Operating Risk.

        submit_btn = st.form_submit_button("🔍 Predict Risk")
        # The form submit button; when clicked, the form values are sent and submit_btn becomes True.

    if submit_btn:
        # This block executes only after the user clicks the form's submit button.

        input_data = np.array([[industrial_risk, management_risk,
                                  financial_flexibility, credibility,
                                  competitiveness, operating_risk]])
        # Build a 2D NumPy array with shape (1, 6), matching the model's expected input shape/order.
        # IMPORTANT: The order must match the feature order used while training the model.

        prediction = model.predict(input_data)[0]
        # Call the model's predict method to get the class label (e.g., 0 or 1). We take [0] to get the single value.

        probability = model.predict_proba(input_data)[0][1] * 100
        # Call predict_proba to get probabilities for each class; [0][1] fetches the probability
        # of the positive class (assumed to be 'bankruptcy'). Multiply by 100 to get a percent.
        # WARNING: If the model does not have predict_proba, this line will raise an AttributeError.

        # Display Result
        st.subheader("Prediction Result")
        # Subheader above the prediction output.

        if prediction == 1:
            st.error(f"⚠️ High Bankruptcy Risk: **{probability:.2f}%**")
            # If predicted class is 1 (bankruptcy), show an error-style message with the percent.
        else:
            st.success(f"✅ Low Bankruptcy Risk: **{probability:.2f}%**")
            # If predicted class is 0 (non-bankruptcy), show a success-style message.

        # Visual Risk Plot
        labels = ["Non-Bankruptcy", "Bankruptcy"]
        # Labels for pie chart slices in the same order as values below.

        values = [100 - probability, probability]
        # Values used in the pie chart: complement and the predicted bankruptcy percentage.

        fig, ax = plt.subplots()
        # Create a Matplotlib figure and axes to draw the pie chart.

        ax.pie(values, labels=labels, autopct="%1.1f%%", startangle=90)
        # Draw a pie chart with percentage labels; startangle rotates the chart for a nicer look.

        st.pyplot(fig)
        # Render the Matplotlib figure in the Streamlit app.

        # Show entered values
        st.write("📋 Entered Values")
        # Title for the small table showing input values.

        input_df = pd.DataFrame(input_data, columns=[
            "Industrial Risk", "Management Risk", "Financial Flexibility",
            "Credibility", "Competitiveness", "Operating Risk"
        ])
        # Convert the input NumPy array to a Pandas DataFrame for pretty display with named columns.

        st.table(input_df)
        # Display the DataFrame as a static table so the user can verify what was sent to the model.

# ----------------------------
# PROJECT INFORMATION PAGE
# ----------------------------
elif menu == "Project Info":
    # If "Project Info" is selected, show details about the project.

    st.title("📘 Project Details")
    # Header for the project info page.

    st.write("""
    **Project Title:** Bankruptcy Prediction using Machine Learning  
    **Course:** Data Science  
    **Organization:** EXCELR  
    **Dataset:** Financial Risk Indicators  
    **Goal:** Build a prediction model that identifies companies at risk of bankruptcy.
    The goal here is to model the probability that a business goes bankrupt from different features.
    The data file contains 7 features about 250 companies
    The data set includes the following variables:
    1.	Industrial_risk: 0=low risk, 0.5=medium risk, 1=high risk.
    2.	Management_risk: 0=low risk, 0.5=medium risk, 1=high risk.
    3.	Financial flexibility: 0=low flexibility, 0.5=medium flexibility, 1=high flexibility.
    4.	Credibility: 0=low credibility, 0.5=medium credibility, 1=high credibility.
    5.	Competitiveness: 0=low competitiveness, 0.5=medium competitiveness, 1=high competitiveness.
    6.	Operating_risk: 0=low risk, 0.5=medium risk, 1=high risk.
    7.	Class: bankruptcy, non-bankruptcy (target variable).
    **Tech Stack Used:**
    - Python, Pandas, NumPy
    - Machine Learning:{1.Logistic Regression,
                        2.Decision Tree,
                        3.Random Forest,
                        4.Support Vector Machine (SVM),
                        5.K-Nearest Neighbors (KNN),
                        6.Gradient Boosting,XGBoost}
    - Deployment: Streamlit
    """)
    # Multiline description of project title, dataset, goal and technologies.

# ABOUT DEVELOPER
elif menu == "About Developer":
    # If "About Developer" is selected, show developer info.

    st.title("👨‍💻 Developer Info")
    # Header for developer info page.

    st.write("""
    **Name:** Amit Choudhury  
    **Role:** Data Scientist  
    **Skills:** Python, ML,SQL, Streamlit,Tableau, Microsoft Office, ChatGPT  
    **Vision:** Creating AI solutions to solve real business challenges 💡
    """)
    # Short bio and skills.

    st.markdown("---")
    # Horizontal rule to separate sections.

    st.write
    #It automatically detects the type of content you give it and displays it properly in the app.

2025-10-28 01:53:49.715 
  command:

    streamlit run C:\Users\Admin\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-28 01:53:49.715 Session state does not function when running a script without `streamlit run`
